tochvision 主要处理图像数据，包含一些常用的数据集、模型、转换函数等。torchvision独立于PyTorch，需要专门安装，torchvision主要包含以下四部分：

torchvision.models: 提供深度学习中各种经典的网络结构、预训练好的模型，如：Alex-Net、VGG、ResNet、Inception等

torchvision.datasets：提供常用的数据集，设计上继承 torch.utils.data.Dataset，主要包括：MNIST、CIFAR10/100、ImageNet、COCO等

torchvision.transforms：提供常用的数据预处理操作，主要包括对Tensor及PIL Image对象的操作

torchvision.utils：工具类，如保存张量作为图像到磁盘，给一个小批量创建一个图像网格

torchvision要注意与pytorch版本和Cuda相匹配。要查询pytorch和torchvision的版本，可以使用下面语句：

In [10]:
import torch
import torchvision
print(torch.__version__)
print(torchvision.__version__)

2.8.0+cu126
0.23.0+cu126


通过 pretrined=True 可以加载预训练模型。pretrained 默认值是 False，不赋值和赋值 False 效果一样

选择了 pretrained 为 True 的话，点击运行会执行下载预训练参数的操作，不加载预训练模型没有这步操作，可以直接使用网络架构

In [46]:
import torchvision.models as models
from torchsummary import summary as sumy

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

resnet18 = models.resnet18(weights='IMAGENET1K_V1')
vgg16 = models.vgg16(weights='IMAGENET1K_V1')
alexnet = models.alexnet(weights='IMAGENET1K_V1')
squeezenet = models.squeezenet1_0(weights='IMAGENET1K_V1')
densenet = models.densenet161(weights='IMAGENET1K_V1')
resnet152 = models.resnet152()
# 可以使用该方法打印加载的预训练模型的参数
for name, param in resnet18.named_parameters():
    print(name, param.shape)
vgg16 = vgg16.to(device)
sumy(vgg16, input_size=(3, 224, 224), device=str(device))


conv1.weight torch.Size([64, 3, 7, 7])
bn1.weight torch.Size([64])
bn1.bias torch.Size([64])
layer1.0.conv1.weight torch.Size([64, 64, 3, 3])
layer1.0.bn1.weight torch.Size([64])
layer1.0.bn1.bias torch.Size([64])
layer1.0.conv2.weight torch.Size([64, 64, 3, 3])
layer1.0.bn2.weight torch.Size([64])
layer1.0.bn2.bias torch.Size([64])
layer1.1.conv1.weight torch.Size([64, 64, 3, 3])
layer1.1.bn1.weight torch.Size([64])
layer1.1.bn1.bias torch.Size([64])
layer1.1.conv2.weight torch.Size([64, 64, 3, 3])
layer1.1.bn2.weight torch.Size([64])
layer1.1.bn2.bias torch.Size([64])
layer2.0.conv1.weight torch.Size([128, 64, 3, 3])
layer2.0.bn1.weight torch.Size([128])
layer2.0.bn1.bias torch.Size([128])
layer2.0.conv2.weight torch.Size([128, 128, 3, 3])
layer2.0.bn2.weight torch.Size([128])
layer2.0.bn2.bias torch.Size([128])
layer2.0.downsample.0.weight torch.Size([128, 64, 1, 1])
layer2.0.downsample.1.weight torch.Size([128])
layer2.0.downsample.1.bias torch.Size([128])
layer2.1.conv1.weight tor

vgg16.state_dict() 前面方法得到模型的参数

加载本地的模型参数使用方法 vgg16.load_state_dict()

需要结合加载文件转化为字典的函数 torch.load_dict() 一起使用，也就是 vgg16.load_state_dict(torch.load_dict(xxxx.pth))

保存模型的参数需要得到模型和的字典，然后使用 torch.save() 函数来保存，torch.save(vgg16.state_dict(), "xxxx.pth")

In [12]:
vgg16_state_dict = vgg16.state_dict()
resnet18_state_dict = resnet18.state_dict()

torch.save(vgg16_state_dict, "./vgg16_state_dict.pth")
torch.save(resnet18_state_dict, "./resnet18_state_dict.pth")

vgg16.load_state_dict(torch.load("./vgg16_state_dict.pth"))

<All keys matched successfully>

但是正常的使用过程中可能会修改模型的层数，想要接着使用预训练的参数就会出现无法加载的问题，所以必须在 pth 文件里面删掉或者加上相应的修改的层

加入现在 resnet18 的分类任务变为了分类为 10 类，就需要对最后的全连接层进行修改

层的名字和 shape 都是匹配的话，成功的加载，名字不匹配但是形状匹配的话，strict=False 会忽略错误，名字匹配但是形状不匹配，都会报错

pretrained_dict = {k : v for k, v in pretrained_dict.items() if k in model_dict} 这个写法很显然只会比较键是不是完全一样，即使修改了形状也不会关心，所以需要加上判断，v.shape 和 model_dict[k].shape 要是一样的才行，这样的话，只会从预训练模型里面加载确定都没有修改的层

In [ ]:
import torch.nn as nn

model = models.resnet18(weights=None)

model.fc = nn.Linear(512, 10)
pretrained_dict = resnet18.state_dict()
model_dict = model.state_dict()
print(pretrained_dict.keys() == model_dict.keys())
print(pretrained_dict['fc.bias'].size())
pretrained_dict = {k : v for k, v in pretrained_dict.items() if k in model_dict and v.shape == model_dict[k].shape}
# print(pretrained_dict['fc.bias']) 这句话会报错因为修改了全连接层，这个参数没有被加载
# res = model.load_state_dict(torch.load('./resnet18_state_dict.pth'), strict=False) 报错，因为无法对应，原本的预训练模型的全连接层的大小为 [512, 1000]，现在被我修改为了 [512, 10]
res = model.load_state_dict(pretrained_dict, strict=False) # 完美的加载，验证了自己的想法，现在删掉了全连接层的参数，其他的参数的预训练的参数了

print("Missing keys:", res.missing_keys) # 模型中存在，但是在预训练参数里面找不到
print("Unexpected keys:", res.unexpected_keys) # 预训练参数中存在模型中没定义，这个不会遇到，因为我强制的比对了键值对，模型中没定义的一定会被删掉的

print(model.fc.weight.mean(), model.fc.weight.std())  # 随机分布
print(resnet18.fc.weight.mean(), resnet18.fc.weight.std())  # 不同分布


True
torch.Size([1000])
Missing keys: ['fc.weight', 'fc.bias']
Unexpected keys: []
tensor(9.4485e-06, grad_fn=<MeanBackward0>) tensor(0.0254, grad_fn=<StdBackward0>)
tensor(5.8502e-08, grad_fn=<MeanBackward0>) tensor(0.0695, grad_fn=<StdBackward0>)


上面简单的介绍了模型的加载和报错，和加载预训练参数的方法等，这个是 torchvision.models 库函数里面常见的操作了，总结下列函数：

models.model() 加载名字为 model 的模型，一般的参数为 weights，代表加载预训练参数，不想加载的话初始化为 weights=None

model.named_parameters() 返回键值对，代表层的名字和对应的 shape

summary() 能将一张指定大小的图片走一遍网络的流程，看一下具体经过一层之后变化为多大的尺寸

torch.save() 保存模型

torch.load_state_dict() 加载模型

torch.load() 加载文件转化为字典

model.state_dict() 得到模型的参数，通过 [键] 能得到对应的值，也就是这个名字的层的大小，也可以直接 name_parameters() 输出键值对，是一样的结果

In [8]:
from torchvision import datasets, transforms as T

transform = T.Compose([
    T.ToTensor(),
    T.Normalize((0.5), (0.5))
])

mnist = datasets.MNIST('data/', download=True, train=False, transform=transform)
# root 数据集的存放路径，没有的话，会自动存放到该文件夹
# train 是否加载训练集，否则的话就是测试集
# transform torchvision.transforms 包含对图像进行变换的函数，ToTensor()、Normalize()
# target_transform torchvision.transforms 对标签变换的函数，一般不使用
# download 如果本地没有数据集，是否从网上下载

img = mnist[0][0]
label = mnist[0][1]
to_img = T.ToPILImage()
display(to_img(img), label)

7

当第一次调用 datasets.MNIST(download=True) 时，它会自动执行以下步骤检查 raw 文件夹如果没有 .gz 文件，就从官网（yann.lecun.com）下载，自动解压 .gz 文件

存入 data/MNIST/processed/，解析二进制数据为 Tensor，把像素值解析成 torch.Tensor 格式，把标签解析成 torch.LongTensor，缓存结果，将解析好的数据保存为两个 .pt 文件

training.pt  → 包含 (train_images, train_labels)，参考上面是输出 dataset 是一个存储文件，保存了 Tensor 类型的文件和对应的标签
test.pt      → 包含 (test_images, test_labels)

因为 datasets.MNIST 继承自 torch.utils.data.Dataset，实现了 \_\_gititem\_\_ 和 \_\_len\_\_ 两个函数，可以直接 train_loader = DataLoader(train_data, batch_size=64, shuffle=True)

不需要进行其他任何多余的操作

In [20]:
from torch.utils.data import DataLoader

train_loader = DataLoader(mnist, batch_size=64, shuffle=True)

train_iter = iter(train_loader)

batch_img, batch_label = next(train_iter)

print(batch_img.shape, batch_label.shape)
for batch_idx, (imgs, labels) in enumerate(train_loader):
    print(batch_idx, imgs.shape, labels.shape)
    break

torch.Size([64, 1, 28, 28]) torch.Size([64])
0 torch.Size([64, 1, 28, 28]) torch.Size([64])


PyTorch 里所有能被 DataLoader 使用的数据集,都必须继承 torch.utils.data.Dataset，并实现这两个方法：\_\_getitem\_\_()：告诉 PyTorch 怎么取一条数据；\_\_len\_\_()：告诉 PyTorch 数据集多大

dataset 对象，加载的数据集，要是自己创建的数据集，需要手动实现上面说的类和类里面必须实现的方法，batch_size 每个批次的样本数量，shuffle 是否打乱数据，num_workers 加载数据的线程数，drop_last 最后一个 batch 不足 batch_size 大小，是否丢弃

train_data.data 所有图片 torch.Tensor (形状 [60000, 28, 28])

train_data.targets 所有标签 torch.Tensor (形状 [60000])

image, label = train_data[0]


In [26]:
from torchvision import transforms as T

# 包含了常见的图像处理的函数

dir(T)

['AugMix',
 'AutoAugment',
 'AutoAugmentPolicy',
 'CenterCrop',
 'ColorJitter',
 'Compose',
 'ConvertImageDtype',
 'ElasticTransform',
 'FiveCrop',
 'GaussianBlur',
 'Grayscale',
 'InterpolationMode',
 'Lambda',
 'LinearTransformation',
 'Normalize',
 'PILToTensor',
 'Pad',
 'RandAugment',
 'RandomAdjustSharpness',
 'RandomAffine',
 'RandomApply',
 'RandomAutocontrast',
 'RandomChoice',
 'RandomCrop',
 'RandomEqualize',
 'RandomErasing',
 'RandomGrayscale',
 'RandomHorizontalFlip',
 'RandomInvert',
 'RandomOrder',
 'RandomPerspective',
 'RandomPosterize',
 'RandomResizedCrop',
 'RandomRotation',
 'RandomSolarize',
 'RandomVerticalFlip',
 'Resize',
 'TenCrop',
 'ToPILImage',
 'ToTensor',
 'TrivialAugmentWide',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_functional_pil',
 '_functional_tensor',
 '_presets',
 'autoaugment',
 'functional',
 'transforms']

Compose()：用来管理所有的 transforms 操作

ToTensor()：把图片数据转换成张量并转化范围在 [0,1] 区间内

Normalize(mean, std)：归一化

Resize(size)：输入的 PIL 图像调整为指定的大小，参数可以为 int 或 int 元组

CenterCrop(size)：将给定的 PIL Image 进行中心切割，得到指定 size 的 tuple

RandomCrop(size, padding=0)：随机中心点切割

RandomHorizontalFlip(size, interpolation=2)：将给定的 PIL Image 随机切割，再 resize

RandomHorizontalFlip()：随机水平翻转给定的 PIL Image

RandomVerticalFlip()：随机垂直翻转给定的 PIL Image

ToPILImage()：将 Tensor 或 numpy.ndarray 转换为 PIL Image

FiveCrop(size)：将给定的 PIL 图像裁剪成 4 个角落区域和中心区域

Pad(padding, fill=0, padding_mode=‘constant’)：对 PIL 边缘进行填充

RandomAffine(degrees, translate=None, scale=None)：保持中心不变的图片进行随机仿射变化

RandomApply(transforms, p=0.5)：随机选取变换


下面自己构建一个 MNIST 类，学习一下一般方法

In [ ]:
from torch.utils.data import Dataset
from tqdm import tqdm
import os
from PIL import Image
import torchvision.transforms as T

class MNIST(Dataset):
    def __init__(self, root, transform=None, preload=False):
        self.samples = []
        self.transform = transform
        self.preload = preload
        self.data = []
        classes = sorted(os.listdir(root))
        for label, cls in enumerate(tqdm(classes, desc="processing", ascii=True)):
            folder = os.path.join(root, cls)
            for img in os.listdir(folder):
                path = os.path.join(folder, img)
                self.samples.append((path, label))
        if self.preload:
            for path, label in self.samples:
                img = Image.open(path).convert('L')
                if self.transform:
                    img = self.transform(img)
                self.data.append((img, label))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, index):
        if self.preload:
            return self.data[index]
        path, label = self.samples[index]
        img = Image.open(path).convert('L')
        if self.transform:
            img = self.transform(img)
        return img, label
transform = T.ToTensor()
dataset = MNIST('./mnist_test_torch', transform=transform, preload=True)





from torch.utils.data import DataLoader

loader = DataLoader(dataset, batch_size=32, shuffle=True)

for imgs, labels in loader: 
    # 执行这句话的时候 __getitem__ 才会执行，相当于进行下面的操作
    # batch_data = []，for idx in [i1, i2, ..., i32]:，sample = dataset.__getitem__(idx)，batch_data.append(sample)
    # 每张图片都要执行一边 getitem，设计图像的解码，打开，转换等操作，本质上是很慢的，它存在的意义就是有人需要对图像进行增强，一张图片可能被改编成很多种的可能，直接存储静态的文件实现不了这一点
    print(imgs.shape, labels.shape)
    break

processing: 100%|##########| 10/10 [00:00<00:00, 501.30it/s]


torch.Size([32, 1, 28, 28]) torch.Size([32])
